In [7]:
import sys
sys.path.append('..')
import ScraperFC as sfc # import local scraperfc
import traceback
from tqdm.auto import tqdm
import pandas as pd

In [2]:
# # from shared_test_functions import get_random_league_seasons

# # samples = get_random_league_seasons('Oddsportal', 3)

# op = sfc.Oddsportal()
# # fbref = sfc.FBRef()
# counts = dict()
# try:
#     # for year, league in tqdm(samples):
#     league = 'EPL'
#     for year in tqdm(range(2004,2023)):
#         # print('-'*80)
#         # print(year, league)
#         # fbref_links = fbref.get_match_links(year,league)
#         op_links = op.get_match_links(year, league)
#         counts[year] = len(op_links)
#         # break
# except:
#     traceback.print_exc()
# finally:
#     op.close()
#     # fbref.close()

In [3]:
op = sfc.Oddsportal()
try:
    # match_link = links[np.random.randint(0, num_links+1)]
    # match_link = 'https://oddsportal.com/football/england/premier-league-2005-2006/chelsea-birmingham-be9APe6e' # no O/U odds
    # match_link = 'https://www.oddsportal.com/football/england/premier-league/west-ham-southampton-tfnoGQAd' # weird date format
    match_link = 'https://www.oddsportal.com/football/england/premier-league/aston-villa-bournemouth-Kdi8EDZi/' # everything should work
    match_df = op.scrape_match(match_link)

    moneyline_df = op.get_1X2odds_from_match(match_link)
    over_under_df = op.get_OUodds_from_match(match_link)
except:
    traceback.print_exc()
finally:
    op.close()

[WDM] - Downloading: 100%|██████████| 6.98M/6.98M [00:00<00:00, 23.5MB/s]


In [ ]:
match_df

In [5]:
ogs = (match_df.copy(), moneyline_df.copy(), over_under_df.copy())

In [26]:
match_df, moneyline_df, over_under_df = ogs

In [27]:
# Format MultiIndices
dfs_to_concat = [
    (match_df, 'Info'), 
    (moneyline_df, '1X2'), 
    (over_under_df, 'O/U')
]
max_levels = max([len(df.columns[0]) if type(df.columns) is not pd.Index else 1 for df,_ in dfs_to_concat])
for df, name in dfs_to_concat:
    current_levels = len(df.columns[0]) if type(df.columns) is not pd.Index else 1

    # Add a name level to isolate different odds types
    name_level = pd.DataFrame([name for i in df.columns], index=df.columns)
    
    # Add empty levels so all MultiIndices have same number of levels
    empty_levels = pd.DataFrame([['']*(max_levels-current_levels) for i in df.columns], index=df.columns)
    
    # Combine all of the columns
    new_columns = pd.MultiIndex.from_frame(pd.concat([name_level, df.columns.to_frame(), empty_levels], axis=1))
    
    # Update columns in df object
    df.columns = new_columns